# Alpha-beta pruning algorithm

### Dependencies

In [ ]:
import chess
import random
import signal
import time

import import_ipynb
import Util

importing Jupyter notebook from Game.ipynb
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
importing Jupyter notebook from Util.ipynb


### get_best_move_alphabeta
Finds the best move to make based on the alpha-beta pruning algorithm. This algorithm works as follows:
- Iterate over all legal moves in the current position.
- For each move, find the best possible score after making this move.
    - This is done by calling the alpha-beta pruning function recursively: increasing the current iteration by 1 and switching turns.
    - If a move yields a worse score than the previously examined move in the hierarchy, it is unlikely that this will ever be the optimal move. Therefore, such moves are not evaluated further.
- Find the maximum (if it's the AI's turn) or the minimum (if it's the player's turn) score of all legal moves, alongside the move that was able to reach this optimized state. This is the move that the algorithm recommends.

Optionally, the algorithm uses memoization, which is a type of caching. This works by mapping a board state, plus iteration details, to the corresponding score and move and storing it in a cache dictionary. If, during a later iteration, the same board state is reached on the same iteration, these values can be read from the cache.

##### Member of class
    chess.Board

##### Arguments
    use_cache: bool
        Argument to decide if memoization should be used or not.
    alpha: int
        The "alpha" value in the current iteration. On the first iteration, this value should be strongly negative.
    beta: int
        The "beta" value in the current iteration. On the first iteration, this value should be strongly positive.
    ai_turn : bool
        Is the current turn of the AI to take?
    iteration: int
        The depth of the search (amount of moves currently looking ahead).
    max_iterations: int
        The maximum depth of the search.

##### Returns
    tuple(best_score, best_move)

    best_score: int
        The board score after making the recommended best move.
    best_move: chess.Move
        The recommended best move to make.

##### Side effects
    - If use_cache is true, the cache is constantly updated with different board states in combination with the best score and move.
    - If the search is interrupted, the board may be in a different state than when the search started.

In [ ]:
alpha_beta_cache = {}

def get_best_move_alphabeta(self, use_cache: bool, alpha: int, beta: int, ai_turn: bool, color: chess.Color,
                            iteration: int, max_iterations: int, last_eval_score: int) -> (int, chess.Move):
                            
    if use_cache and (iteration, max_iterations, self.get_state_string()) in alpha_beta_cache:
        return alpha_beta_cache[(iteration, max_iterations, self.get_state_string())]

    result = self.get_search_result_if_finished(iteration, max_iterations, last_eval_score)
    if result is not None: return result

    # If the game has not finished, check additional moves using the alpha-beta pruning algorithm
    best_score, best_move = 10000000 * (-1 if ai_turn else 1), None
    for move in self.legal_moves:
        eval_score = self.evaluate_move(color, not ai_turn, last_eval_score, move)
        self.push(move)
        score_after_move, _ = self.get_best_move_alphabeta(use_cache, alpha, beta, not ai_turn, not color, iteration + 1,
                                                           max_iterations, eval_score)
        if (ai_turn and score_after_move > best_score) or (not ai_turn and score_after_move < best_score):
            best_score = score_after_move
            best_move = move
        self.pop()

        if ai_turn:
            if best_score >= beta:
                return best_score, move
            if best_score > alpha:
                alpha = best_score
        else:
            if best_score <= alpha:
                return best_score, move
            if best_score < beta:
                beta = best_score

    if use_cache: alpha_beta_cache[(iteration, max_iterations, self.get_state_string())] = best_score, best_move
    return best_score, best_move

chess.Board.get_best_move_alphabeta = get_best_move_alphabeta

---
<br>

## OutOfTimeError class
A custom exception class that should be raised if a time limit is exceeded.
<br><br>

### Class variables
    N/A
<br>

### \_\_init\_\_
    The constructor for the OutOfTimeError class. Gets called whenever a new OutOfTimeError instance is created (most commonly, when an OutOfTimeError is raised).
<br>

##### Arguments
    message : str (optional)
        A message explaining the raised exception.

##### Returns
    N/A

##### Side effects
    N/A

In [ ]:
class OutOfTimeError(Exception):
    def __init__(self, message='Out of time'):
        self.message = message
        super().__init__(self.message)

### time_limit_signal_handler
Handles the signal that is created when a time limit is exceeded by raising an OutOfTimeError.

##### Arguments
    signum: int
        The signal number corresponding to the signal that triggered this handler.
    frame: signal.frame
        The current stack frame as the signal is triggered.

##### Returns
    N/A

##### Side effects
    - An OutOfTimeError is raised.

In [ ]:
def time_limit_signal_handler(signum, frame):
    raise OutOfTimeError()

### get_best_move_alphabeta_iterative_deepening
Performs an alpha-beta search using the previously described algorithm. However, this function uses iterative deepening, which means that it iterates over a range between 1 and a given maximum depth limit, and on each iteration, it finds the best move using an alpha-beta search with the current maximum depth. Essentially, it moves through the search tree horizontally, rather than hierarchically.

The advantage of this method is that the search can be interrupted at any time, yielding the best move at the previously evaluated depth. Enabling memoization is strongly recommended to reach acceptable performance.

The search stops if either the maximum depth is reached, or if the time limit is exceeded, whichever occurs first.

##### Member of class
    chess.Board

##### Arguments
    use_cache: bool
        Argument to decide if memoization should be used or not.
    max_iterations: int
        The maximum depth of the search.
    time_limit: int
        The maximum duration of the search, in seconds.

##### Returns
    tuple(best_score, best_move)

    best_score: int
        The board score after making the recommended best move.
    best_move: chess.Move
        The recommended best move to make.

##### Side effects
    - If use_cache is true, the cache is constantly updated with different board states in combination with the best score and move.
    - If the search is interrupted, the board may be in a different state than when the search started.

In [ ]:
def get_best_move_alphabeta_iterative_deepening(self, use_cache: bool, color: chess.Color, max_iterations: int, time_limit: int) -> chess.Move:
    signal.signal(signal.SIGALRM, time_limit_signal_handler)
    signal.alarm(time_limit)

    best_move = None
    board_state = self.fen() # Save state, because the board may be in a different state if the search is interrupted

    try:
        for current_max_iterations in range(1, max_iterations + 1):
            _, best_move = self.get_best_move_alphabeta(
                use_cache=use_cache,
                alpha=-100000000,
                beta=100000000,
                ai_turn=True,
                color=color,
                iteration=0,
                max_iterations=current_max_iterations,
                last_eval_score=0 # The starting board score doesn't matter, it's evaluated by score difference
            )
    except OutOfTimeError:
        # The exception will trigger after a fixed amount of time
        self.set_fen(board_state) # Reset the board state because the search was interrupted, leaving the board in a different state

    return best_move

chess.Board.get_best_move_alphabeta_iterative_deepening = get_best_move_alphabeta_iterative_deepening

---
<br>

## make_move_alphabeta

This function gets the best possible move according to the alpha-beta pruning algorithm and pushes it onto the move stack.

#### Arguments
    board: chess.Board
        The board to push the move to.
#### Returns
    N/A
#### Side effects
    - The best possible move is pushed to the move stack of the board.

In [ ]:
def make_move_alphabeta(board: chess.Board, color: chess.Color) -> None:
    _, move = board.get_best_move_alphabeta(
        use_cache=True, 
        alpha=-100000000, 
        beta=100000000, 
        ai_turn=True, 
        color=color,
        iteration=0, 
        max_iterations=5,
        last_eval_score=0 # The starting board score doesn't matter, it's evaluated by score difference
    )
    
    board.push(move)

---
<br>

## make_move_alphabeta_iterative_deepening

This function gets the best possible move according to the alpha-beta pruning algorithm, with iterative deepening, and pushes it onto the move stack.

#### Arguments
    board: chess.Board
        The board to push the move to.
#### Returns
    N/A
#### Side effects
    - The best possible move is pushed to the move stack of the board.

In [ ]:
def make_move_alphabeta_iterative_deepening(board: chess.Board, color: chess.Color) -> None:
    move = board.get_best_move_alphabeta_iterative_deepening(use_cache=True, color=color, max_iterations=10, time_limit=30)
    board.push(move)

In [ ]:
##game = Game.Gam2e(make_move_alphabeta_iterative_deepening)
#game.play()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>